In [20]:

#Importing libraries
import numpy as np
import pandas as pd 
import os


In [21]:
#Creating directory for final data if not already created

if os.path.isdir('./data') == False:
    os.mkdir('./data')
    

In [22]:

#Getting all files names in raw_data folder and storing as list
files = []
for (dirpath, dirnames, filenames) in os.walk('./raw_data'):
    files.extend(filenames)
    break

print(files)

#Reading in all files in raw_data as a dataframe and storing in a list
raw_data_list = [] 
df = pd.DataFrame()
for each in files:
    df = pd.read_csv('./raw_data/' + each)
    raw_data_list.append(df)


# tmp_list = [raw_data_list[0]]




['BTC_day_test.csv', 'BTC_day_train.csv', 'BTC_hour_test.csv', 'BTC_hour_train.csv', 'BTC_min_test.csv', 'BTC_min_train.csv', 'ETH_day_test.csv', 'ETH_day_train.csv', 'ETH_hour_test.csv', 'ETH_hour_train.csv', 'ETH_min_test.csv', 'ETH_min_train.csv']


In [23]:

inc = 0

#Iterating through all dataframes in raw_data_list
for each in raw_data_list:

    dp_list = []
    dp_list.append(None)
    for row in range(1, each.shape[0]):
        delta_P = each['Close'].iloc[row] - each['Close'].iloc[row - 1]
        dp_list.append(delta_P)
    
    #Calculating change in price
    each['DeltaPrice'] = dp_list
    
    #Min Max Scaling change in price
    # maxDeltaPrice = max([abs(each['DeltaPrice'].max()), abs(each['DeltaPrice'].min())])
    # each['DeltaPriceScaled'] = each['DeltaPrice']/maxDeltaPrice
    
    #Calulating Zscores with a 14 day moving average
    closePriceList = each['Close'].tolist()
    
    std_dev_14 = []
    for i in range(0, len(closePriceList)):
        if (0 <= i <= 13):
            std_dev_14.append(None)
            continue
        temp_list = []
        for j in range(i - 14, i):
            temp_list.append(closePriceList[j])
            
        mean = np.mean(temp_list)
        std_dev = np.std(temp_list)
        obs_val = closePriceList[i]
        z = (obs_val - mean)/std_dev
        
        std_dev_14.append(z)
            
    each['Zscore'] = std_dev_14 
    
    # #Min Max Scaling zscores
    # maxZscore = max([abs(each['Zscore'].max()), abs(each['Zscore'].min())])
    # each['ZscoreScaled'] = each['Zscore']/maxZscore
    
    # #Min Max Scaling count
    # maxCount = max([abs(each['Count'].max()), abs(each['Count'].min())])
    # each['CountScaled'] = each['Count']/maxCount
    
    # #Min Max Scaling close
    # maxClose = max([abs(each['Close'].max()), abs(each['Close'].min())])
    # each['CloseScaled'] = each['Close']/maxClose
    
    # #Min Max Scaling Volume
    # maxVolume = max([abs(each['Volume'].max()), abs(each['Volume'].min())])
    # each['VolumeScaled'] = each['Volume']/maxVolume
    
    # #Min Max Scaling VWAP
    # maxVWAP = max([abs(each['VWAP'].max()), abs(each['VWAP'].min())])
    # each['VWAP_Scaled'] = each['VWAP']/maxVWAP
    
    #Dropping some no longer needed columns
    each = each.drop(columns=[each.columns[0], 'timestamp', 'Asset_ID', 'Open', 'High', 'Low'])
    
    # #Creating Target values
    # store = []
    # store = each['DeltaPrice']
    
    # each['Target'] = store
    # each['Target'] = each['Target'].shift(-1)
    # each['Target'].append(None)
    
    
    #Dropping first 14 rows and last rows because they contain NaN
    each = each.drop(range(0,14))
    # each.drop(each.tail(1).index,inplace=True)
    
    #Reseting dataframe index
    each = each.reset_index(drop=True)
    
    # print(each)
    
    #Exporting all finished dataframes to cvs files in data folder
    each.to_csv('./data/' + files[inc])
    # each.to_csv('test.csv')
    inc = inc + 1
    
    
    

    